In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import time
import math
from pathlib import Path 

In [14]:
pd.set_option('display.max_columns',None)
data = pd.read_csv('wnba_data_set.csv')
# valores nulos
data.isna().sum()
#limpa valores nulos
data = data.fillna(value=0)

In [15]:
data.columns

Index(['SEASON', 'SEASON_TYPE', 'PLAYER_ID', 'RANK', 'PLAYER', 'TEAM_ID',
       'TEAM', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
       'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'EFF', 'AST_TOV', 'STL_TOV'],
      dtype='object')

In [16]:
data = data.drop(columns = ["RANK","EFF"])


In [17]:
# per game, using the parameter created in the nba file

total_cols = ["GP",'MIN',"FGM","FGA","FG3M","FG3A",'FTM',"FTA","OREB","DREB","REB",
              "AST","STL","BLK","TOV","PF","PTS",'PLAYERS']
data["PLAYERS"] = 1
datas = list(data['SEASON'].unique())
wnba_data_per_team_by_reg_season = pd.DataFrame(columns=total_cols)
time_st = time.time()
for date in datas:
    temp_table = data[(data['SEASON']==date)&(data['SEASON_TYPE']=='Regular%20Season')].groupby(['TEAM'])[total_cols].sum().reset_index()
    temp_table["POSSETIONS"] = temp_table["FGA"] - temp_table["OREB"] + temp_table["TOV"] + 0.44*temp_table["FTA"]
    temp_table = temp_table[list(temp_table.columns[0:-2])+['POSSETIONS']+["PLAYERS"]]
    temp_table['RGP'] = (temp_table["MIN"]/(40*5))
    for col in temp_table.columns[2:-2]:
        temp_table[col] = (temp_table[col]/temp_table["RGP"])
    nm_2 = temp_table["POSSETIONS"].mean()
    temp_table["TIME/POSS"] = (((temp_table["POSSETIONS"]/(nm_2+temp_table["POSSETIONS"]))*(temp_table["MIN"]/(5)))/(temp_table["POSSETIONS"]))*60
    temp_table["SEASON_START_YEAR"] = date
    wnba_data_per_team_by_reg_season = pd.concat([wnba_data_per_team_by_reg_season,temp_table],axis=0)

#there's a problem with this dataset, in the year 2003 the cleveland rockers has only one players, so this makes this team
#you can see with this filter, data[(data['SEASON_TYPE']=='Regular%20Season') & (data["SEASON"]==2003) & (data["TEAM"]=='CLE')]
#irrelevant to the analysis, it will only damage the data, so im gonna take it out in the next table
#i will continue to exist in the data dataframe


indexer = []
for i in range(0,len(wnba_data_per_team_by_reg_season)):
    indexer.append(i)

wnba_data_per_team_by_reg_season["Index"] = indexer
wnba_data_per_team_by_reg_season.set_index('Index',inplace=True)

wnba_data_per_team_by_reg_season["SEASON_START_YEAR"] = wnba_data_per_team_by_reg_season["SEASON_START_YEAR"].astype('int64')

#removing row, as said before
row_to_drop = list(wnba_data_per_team_by_reg_season[(wnba_data_per_team_by_reg_season["SEASON_START_YEAR"]==2003) & (wnba_data_per_team_by_reg_season["TEAM"]=='CLE')].index) 
wnba_data_per_team_by_reg_season.drop(row_to_drop,inplace=True)
indexer.remove(row_to_drop[0])
#reworking the indexes
indexer = []
for i in range(0,len(wnba_data_per_team_by_reg_season)):
    indexer.append(i)

wnba_data_per_team_by_reg_season["Index"] = indexer
wnba_data_per_team_by_reg_season.set_index('Index',inplace=True)

wnba_data_per_team_by_reg_season = wnba_data_per_team_by_reg_season[["SEASON_START_YEAR"]+["TEAM"]+list(wnba_data_per_team_by_reg_season.columns[0:-5])+list(wnba_data_per_team_by_reg_season.columns[-4:-1])]
wnba_data_per_team_by_reg_season["FG%"] = wnba_data_per_team_by_reg_season["FGM"]/wnba_data_per_team_by_reg_season["FGA"]


for x in range(0,len(wnba_data_per_team_by_reg_season)):
    if wnba_data_per_team_by_reg_season.loc[x,"FG3A"] !=0:
        wnba_data_per_team_by_reg_season.loc[x,"FG3%"] = wnba_data_per_team_by_reg_season.loc[x,"FG3M"]/wnba_data_per_team_by_reg_season.loc[x,"FG3A"]
    else:
        wnba_data_per_team_by_reg_season.loc[x,"FG3%"] = 0

wnba_data_per_team_by_reg_season["FT%"] = wnba_data_per_team_by_reg_season["FTM"]/wnba_data_per_team_by_reg_season["FTA"]
wnba_data_per_team_by_reg_season["FG3A%"] = wnba_data_per_team_by_reg_season["FG3A"]/wnba_data_per_team_by_reg_season["FGA"]
wnba_data_per_team_by_reg_season["PTS/FGA"] = wnba_data_per_team_by_reg_season["PTS"]/wnba_data_per_team_by_reg_season["FGA"]
wnba_data_per_team_by_reg_season["FG3M/FGM"] = wnba_data_per_team_by_reg_season["FG3M"]/wnba_data_per_team_by_reg_season["FGM"]
wnba_data_per_team_by_reg_season["FTA/FGA"] = wnba_data_per_team_by_reg_season["FTA"]/wnba_data_per_team_by_reg_season["FGA"]
wnba_data_per_team_by_reg_season["TRU%"] = 0.5*wnba_data_per_team_by_reg_season["PTS"]/(wnba_data_per_team_by_reg_season["FGA"]+0.44*wnba_data_per_team_by_reg_season["FTA"])
wnba_data_per_team_by_reg_season["AST_TOV"] = wnba_data_per_team_by_reg_season["AST"]/wnba_data_per_team_by_reg_season["TOV"]
wnba_data_per_team_by_reg_season["eFG%"] = (wnba_data_per_team_by_reg_season["FGM"]+wnba_data_per_team_by_reg_season["FG3M"]*0.5)/wnba_data_per_team_by_reg_season["FGA"]
wnba_data_per_team_by_reg_season["ISO%"] = (wnba_data_per_team_by_reg_season["FGM"]-wnba_data_per_team_by_reg_season["AST"])/wnba_data_per_team_by_reg_season["FGM"]
wnba_data_per_team_by_reg_season["FG2%"] = (wnba_data_per_team_by_reg_season["FGM"]-wnba_data_per_team_by_reg_season["FG3M"])/(wnba_data_per_team_by_reg_season["FGA"]-wnba_data_per_team_by_reg_season["FG3A"])


print(f"Finished in {time.time()-time_st}") 


Finished in 0.3029208183288574


In [18]:
#this houston in 2008 is also really strange
#wnba_data_per_team_by_reg_season[wnba_data_per_team_by_reg_season["GP"]<100]
#data[(data['SEASON_TYPE']=='Regular%20Season') & (data["SEASON"]==2008) & (data["TEAM"]=='HOU')]

In [19]:
## create per 100 possetions to compare with the nba dataset

In [20]:
## per 100

per_100_cols = ['SEASON_START_YEAR',
 'RGP',
 'TEAM',
 'PLAYERS',
 'POSSETIONS',
 'TIME/POSS',
 'FGM',
 'FGA',
 'FG3M',
 'FG3A',
 'FTM',
 'FTA',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS']
wnba_data_per100_team_by_reg_season = wnba_data_per_team_by_reg_season[per_100_cols].copy()
for col in wnba_data_per100_team_by_reg_season.columns[6:]:
    wnba_data_per100_team_by_reg_season[col]=(wnba_data_per100_team_by_reg_season[col]/wnba_data_per100_team_by_reg_season["POSSETIONS"])*100
    
wnba_data_per100_team_by_reg_season.rename(columns={'POSSETIONS':'POSSETIONS_PER_GAME'},inplace=True)
wnba_data_per100_team_by_reg_season["FG%"] = wnba_data_per100_team_by_reg_season["FGM"]/wnba_data_per100_team_by_reg_season["FGA"]
for x in range (0,len(wnba_data_per100_team_by_reg_season)):
    if wnba_data_per100_team_by_reg_season.loc[x,"FG3A"] != 0:
        wnba_data_per100_team_by_reg_season.loc[x,"FG3%"] = wnba_data_per100_team_by_reg_season.loc[x,"FG3M"]/wnba_data_per100_team_by_reg_season.loc[x,"FG3A"]
    else:
        wnba_data_per100_team_by_reg_season.loc[x,"FG3%"] = 0
        
wnba_data_per100_team_by_reg_season["FT%"] = wnba_data_per100_team_by_reg_season["FTM"]/wnba_data_per100_team_by_reg_season["FTA"]
wnba_data_per100_team_by_reg_season["FG3A%"] = wnba_data_per100_team_by_reg_season["FG3A"]/wnba_data_per100_team_by_reg_season["FGA"]
wnba_data_per100_team_by_reg_season["PTS/FGA"] = wnba_data_per100_team_by_reg_season["PTS"]/wnba_data_per100_team_by_reg_season["FGA"]
wnba_data_per100_team_by_reg_season["FG3M/FGM"] = wnba_data_per100_team_by_reg_season["FG3M"]/wnba_data_per100_team_by_reg_season["FGM"]
wnba_data_per100_team_by_reg_season["FTA/FGA"] = wnba_data_per100_team_by_reg_season["FTA"]/wnba_data_per100_team_by_reg_season["FGA"]
wnba_data_per100_team_by_reg_season["TRU%"] = 0.5*wnba_data_per100_team_by_reg_season["PTS"]/(wnba_data_per100_team_by_reg_season["FGA"]+0.44*wnba_data_per100_team_by_reg_season["FTA"])
wnba_data_per100_team_by_reg_season["AST_TOV"] = wnba_data_per100_team_by_reg_season["AST"]/wnba_data_per100_team_by_reg_season["TOV"]
wnba_data_per100_team_by_reg_season["eFG%"] = (wnba_data_per100_team_by_reg_season["FGM"]+wnba_data_per100_team_by_reg_season["FG3M"]*0.5)/wnba_data_per100_team_by_reg_season["FGA"]
wnba_data_per100_team_by_reg_season["ISO%"] = (wnba_data_per100_team_by_reg_season["FGM"]-wnba_data_per100_team_by_reg_season["AST"])/wnba_data_per100_team_by_reg_season["FGM"]
wnba_data_per100_team_by_reg_season["FG2%"] = (wnba_data_per100_team_by_reg_season["FGM"]-wnba_data_per100_team_by_reg_season["FG3M"])/(wnba_data_per100_team_by_reg_season["FGA"]-wnba_data_per100_team_by_reg_season["FG3A"])



In [21]:
filepath = Path('C:/Users/vinic/Documents/Python/NBA/wnba/wnba_reg_season_team_per100_data_set.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
wnba_data_per100_team_by_reg_season.to_csv(filepath,index=False)

filepath = Path('C:/Users/vinic/Documents/Python/NBA/wnba/wnba_reg_season_team_per_game_data_set.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
wnba_data_per_team_by_reg_season.to_csv(filepath,index=False)


In [22]:
## creating per season per 100 pos
cols_1 = ['RGP', 'POSSETIONS_PER_GAME', 'TIME/POSS']
cols = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
number_of_teams = wnba_data_per100_team_by_reg_season.SEASON_START_YEAR.value_counts()
to_join = wnba_data_per100_team_by_reg_season.groupby(["SEASON_START_YEAR"])[cols_1].sum().reset_index()
wnba_data_per100_per_season_by_reg_season = wnba_data_per100_team_by_reg_season.groupby(["SEASON_START_YEAR"])[cols].sum().reset_index()
wnba_data_per100_per_season_by_reg_season = to_join.join(wnba_data_per100_per_season_by_reg_season.set_index('SEASON_START_YEAR'),on="SEASON_START_YEAR")
wnba_data_per100_per_season_by_reg_season["Number_of_teams"] = 0
for y in number_of_teams.index:
    position = wnba_data_per100_per_season_by_reg_season[wnba_data_per100_per_season_by_reg_season["SEASON_START_YEAR"]==y].index[0]
    value = number_of_teams.loc[y]
    wnba_data_per100_per_season_by_reg_season.loc[position,"Number_of_teams"] = value
for col in wnba_data_per100_per_season_by_reg_season.columns[1:-1]:
    wnba_data_per100_per_season_by_reg_season[col] = wnba_data_per100_per_season_by_reg_season[col]/wnba_data_per100_per_season_by_reg_season["Number_of_teams"]
wnba_data_per100_per_season_by_reg_season["FG%"] = wnba_data_per100_per_season_by_reg_season["FGM"]/wnba_data_per100_per_season_by_reg_season["FGA"]
for x in range (0,len(wnba_data_per100_per_season_by_reg_season)):
    if wnba_data_per100_per_season_by_reg_season.loc[x,"FG3A"] != 0:
        wnba_data_per100_per_season_by_reg_season.loc[x,"FG3%"] = wnba_data_per100_per_season_by_reg_season.loc[x,"FG3M"]/wnba_data_per100_per_season_by_reg_season.loc[x,"FG3A"]
    else:
        wnba_data_per100_per_season_by_reg_season.loc[x,"FG3%"] = 0
        
wnba_data_per100_per_season_by_reg_season["FT%"] = wnba_data_per100_per_season_by_reg_season["FTM"]/wnba_data_per100_per_season_by_reg_season["FTA"]
wnba_data_per100_per_season_by_reg_season["FG3A%"] = wnba_data_per100_per_season_by_reg_season["FG3A"]/wnba_data_per100_per_season_by_reg_season["FGA"]
wnba_data_per100_per_season_by_reg_season["PTS/FGA"] = wnba_data_per100_per_season_by_reg_season["PTS"]/wnba_data_per100_per_season_by_reg_season["FGA"]
wnba_data_per100_per_season_by_reg_season["FG3M/FGM"] = wnba_data_per100_per_season_by_reg_season["FG3M"]/wnba_data_per100_per_season_by_reg_season["FGM"]
wnba_data_per100_per_season_by_reg_season["FTA/FGA"] = wnba_data_per100_per_season_by_reg_season["FTA"]/wnba_data_per100_per_season_by_reg_season["FGA"]
wnba_data_per100_per_season_by_reg_season["TRU%"] = 0.5*wnba_data_per100_per_season_by_reg_season["PTS"]/(wnba_data_per100_per_season_by_reg_season["FGA"]+0.44*wnba_data_per100_per_season_by_reg_season["FTA"])
wnba_data_per100_per_season_by_reg_season["AST_TOV"] = wnba_data_per100_per_season_by_reg_season["AST"]/wnba_data_per100_per_season_by_reg_season["TOV"]
wnba_data_per100_per_season_by_reg_season["eFG%"] = (wnba_data_per100_per_season_by_reg_season["FGM"]+wnba_data_per100_per_season_by_reg_season["FG3M"]*0.5)/wnba_data_per100_per_season_by_reg_season["FGA"]
wnba_data_per100_per_season_by_reg_season["ISO%"] = (wnba_data_per100_per_season_by_reg_season["FGM"]-wnba_data_per100_per_season_by_reg_season["AST"])/wnba_data_per100_per_season_by_reg_season["FGM"]
wnba_data_per100_per_season_by_reg_season["FG2%"] = (wnba_data_per100_per_season_by_reg_season["FGM"]-wnba_data_per100_per_season_by_reg_season["FG3M"])/(wnba_data_per100_per_season_by_reg_season["FGA"]-wnba_data_per100_per_season_by_reg_season["FG3A"])



In [23]:
fig = go.Figure()

for col in wnba_data_per100_per_season_by_reg_season.columns[1:]:
    fig.add_trace(go.Scatter(x=wnba_data_per100_per_season_by_reg_season['SEASON_START_YEAR'],y=wnba_data_per100_per_season_by_reg_season[col],name=col))

fig.update_layout(
            title={
            'text' : "WNBA During the Seasons per 100 Possetions",
            'x':0.5,
            'xanchor': 'center'
        })
fig.show()

In [24]:
filepath = Path('C:/Users/vinic/Documents/Python/NBA/wnba/wnba_reg_season_season_per100_data_set.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
wnba_data_per100_per_season_by_reg_season.to_csv(filepath,index=False)